In [82]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [83]:
df_pos = pd.read_excel('data/output/clean_handlabeled_data.xlsx')
df_neg = pd.read_excel('data/output/clean_handlabeled_data.xlsx')

Raghava

In [84]:
df_bit = pd.read_excel("data/output/clean_bit_data.xlsx") 

In [85]:
df_bit.head()

,date,COMMENT,LABEL
0,04-05-19,"hold, optimisticvalue, becomes, distinct, poss...",NaN
1,04-05-19,"thumb, nasty, bear, love, get, spank, cryptoland",NaN
2,04-05-19,"mysterious, crypto, whale, unveils, meteoric, ...",NaN
3,04-05-19,"probably, pessimisticvalue, monday",NaN
4,04-05-19,"short, btc, maßsive, profit",NaN


## Classifier

In [86]:
df_pos['LABEL'] = df_pos['LABEL'].map({'NEU':'POS','POS':'POS','NEG':'NEG'})

In [87]:
df_neg['LABEL'] = df_neg['LABEL'].map({'NEU':'NEG','POS':'POS','NEG':'NEG'})

In [88]:
def classifier(clf,strat,vec,df):
    
    y = df['LABEL'].astype(str)
    X = df.COMMENT.astype(str)
    #y = pd.get_dummies(y)
    
    if vec == "cv":
        count = CountVectorizer()
    elif vec == "tdidf":
        count = TfidfVectorizer(binary=True, ngram_range=(1,1), use_idf=False)
    
    if strat == "y":
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        X_train_counts = count.fit_transform(X_train)

        X_test_counts = count.transform(X_test)

        
    elif strat == "n":
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
        X_train_counts = count.fit_transform(X_train)
        X_test_counts = count.transform(X_test)

    
    if clf == "lr":
        clf = LogisticRegression(solver="newton-cg",C=1).fit(X_train_counts, y_train)
        #clf = LogisticRegression(C=0.4).fit(X_train_counts, y_train)
        
    elif clf == "mnb":
        clf = MultinomialNB().fit(X_train_counts, y_train)
        
    
    elif clf == "svm":
        clf = svm.SVC(kernel='linear').fit(X_train_counts, y_train)

    yPred = clf.predict(X_test_counts)
    
    list_ = list(zip(yPred, X_test))
    
    acc=accuracy_score(y_test,yPred)
    f1=f1_score(y_test,yPred, average='weighted')

    return clf, count,"Accuracy: %s" % acc, "F1 %s" % f1, list_


In [89]:
result=classifier("lr","y","cv",df_pos)
print(result[2:4])

df_result = pd.DataFrame(result[4], columns = ['Predicted_label', 'Comment'])

('Accuracy: 0.63', 'F1 0.614927617551341')


In [90]:
result[0].classes_

array(['NEG', 'POS'], dtype=object)

Trying Pipeline

In [92]:
df_p=df_bit.iloc[:5]

In [93]:
#df_p['COMMENT'] = df_p['COMMENT'].dropna()

df_p.head()

,date,COMMENT,LABEL
0,04-05-19,"hold, optimisticvalue, becomes, distinct, poss...",NaN
1,04-05-19,"thumb, nasty, bear, love, get, spank, cryptoland",NaN
2,04-05-19,"mysterious, crypto, whale, unveils, meteoric, ...",NaN
3,04-05-19,"probably, pessimisticvalue, monday",NaN
4,04-05-19,"short, btc, maßsive, profit",NaN


In [94]:
feature_to_coef = {
    word: coef for word, coef in zip(
        result[1].get_feature_names(), result[0].coef_[0]
    )    
}

['hold', 'optimisticvalue', 'becomes', 'distinct', 'possibility', 'real', 'permabears']
['thumb', 'nasty', 'bear', 'love', 'get', 'spank', 'cryptoland']
['mysterious', 'crypto', 'whale', 'unveils', 'meteoric', 'bitcoin', 'price', 'target']
['probably', 'pessimisticvalue', 'monday']
['short', 'btc', 'maßsive', 'profit']


/Users/morten/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [81]:
df_p

,date,COMMENT,LABEL
0,04-05-19,"hold, optimisticvalue, becomes, distinct, poss...",NaN
1,04-05-19,"thumb, nasty, bear, love, get, spank, cryptoland",NaN
2,04-05-19,"mysterious, crypto, whale, unveils, meteoric, ...",NaN
3,04-05-19,"probably, pessimisticvalue, monday",NaN
4,04-05-19,"short, btc, maßsive, profit",NaN
5,04-05-19,"many, btc, u, buy, da, dip, lot",NaN
6,04-05-19,"wall, street, analyst, scream, ‘, buy, bitcoin...",NaN
7,04-05-19,NaN,NaN
8,04-05-19,"optimisticvalue, resistance, scalper",NaN
9,04-05-19,"anyone, else, hear, jaw, theme, play, backgrou...",NaN


In [854]:
filter(lambda comment: comment[0] == comment[0], feature_to_coef)

In [ ]:
for dict_ in [x for x in list_dicts if x["key3"] == 20]:
        pass

In [760]:
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], reverse=True, 
    )[:10]:
    print (best_positive)

('optimisticvalue', 1.2436309459467882)
('nan', 1.1704014718232685)
('lol', 1.1200885697063214)
('bitcoin', 0.9272247641098809)
('hold', 0.9229540254955647)
('chart', 0.9108381893021451)
('kumar', 0.8847881796181486)
('anyone', 0.8482014771180382)
('currently', 0.8267499671782019)
('data', 0.8095678148284611)


In [761]:
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1] 
    )[:10]:
    print (best_negative)

('pessimisticvalue', -1.569137812737998)
('level', -1.152981476058281)
('low', -1.132834752121932)
('without', -1.0455949767618766)
('sell', -1.0405330179427672)
('mad', -0.9303355448258249)
('crash', -0.9299573994425844)
('wouldnt', -0.9262072239074012)
('investor', -0.8996887248666484)
('recover', -0.8513531615290596)
